<a href="https://colab.research.google.com/github/Debanjand20/C-PROJECT-2ND-SEM/blob/main/Previous%20Year%20Question%20Paper%20Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install streamlit google-api-python-client google-auth sentence-transformers spacy
!python -m spacy download en_core_web_sm



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [32]:
from google.colab import files
uploaded = files.upload()

# Rename it for use
import shutil
shutil.move("service_account.json", "/content/service_account.json")


Saving service_account.json to service_account (1).json


'/content/service_account.json'

In [39]:
%%writefile gdrive_utils.py
import re  # Ensure this import is at the very top
from google.oauth2 import service_account
from googleapiclient.discovery import build

def get_drive_service():
    creds = service_account.Credentials.from_service_account_file(
        "/content/service_account.json",
        scopes=["https://www.googleapis.com/auth/drive.readonly"]
    )
    return build("drive", "v3", credentials=creds)

def list_pdf_metadata(service, root_folder_id, requested_year):
    metadata = []

    def list_folders(parent_id):
        folders = []
        page_token = None
        while True:
            response = service.files().list(
                q=f"'{parent_id}' in parents and mimeType='application/vnd.google-apps.folder'",
                fields="nextPageToken, files(id, name)",
                pageSize=1000,
                pageToken=page_token
            ).execute()
            folders.extend(response.get("files", []))
            page_token = response.get("nextPageToken")
            if not page_token:
                break
        return folders

    def list_pdfs(folder_id):
        pdfs = []
        page_token = None
        while True:
            response = service.files().list(
                q=f"'{folder_id}' in parents and mimeType='application/pdf'",
                fields="nextPageToken, files(id, name)",
                pageSize=1000,
                pageToken=page_token
            ).execute()
            pdfs.extend(response.get("files", []))
            page_token = response.get("nextPageToken")
            if not page_token:
                break
        return pdfs

    # Step 1: Subject folders (e.g., Bengali, History, etc.)
    subjects = list_folders(root_folder_id)

    for subj in subjects:
        subj_name = subj["name"]
        subj_id = subj["id"]

        # Step 2: Gen / Hons under each subject
        courses = list_folders(subj_id)

        for course in courses:
            course_name = course["name"]
            course_id = course["id"]

            # Step 3: Semester folders under Gen/Hons
            sem_folders = list_folders(course_id)

            for sem in sem_folders:
                sem_name = sem["name"]
                sem_id = sem["id"]

                # Step 4: PDF files in sem folder
                pdfs = list_pdfs(sem_id)

                for pdf in pdfs:
                    # Extract year from the PDF name using regex
                    year_match = re.search(r'(\d{4})', pdf["name"])
                    year = int(year_match.group(1)) if year_match else None

                    # Only include PDF if the year matches
                    if requested_year is None or (year == requested_year):
                        metadata.append({
                            "id": pdf["id"],
                            "name": pdf["name"],
                            "subject": subj_name,
                            "course": course_name,
                            "semester_folder": sem_name,
                            "year": year
                        })

    return metadata

def file_link(file_id):
    return f"https://drive.google.com/uc?export=download&id={file_id}"




Overwriting gdrive_utils.py


In [46]:
%%writefile parse_query.py
import re
import spacy
from sentence_transformers import SentenceTransformer, util

# Load NLP model
nlp = spacy.load("en_core_web_sm")

# Load sentence transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Extract year, sem, subject, course from query
def parse_query(text):
    doc = nlp(text)
    result = {"year": None, "sem": None, "subject": None, "course": None}

    # Year
    for ent in doc.ents:
        if ent.label_ == "DATE" and ent.text.isdigit():
            result["year"] = int(ent.text)

    # Semester
    for i, token in enumerate(doc):
        if token.text.lower() in {"sem", "semester"}:
            if i + 1 < len(doc) and doc[i + 1].like_num:
                result["sem"] = int(doc[i + 1].text)

    # Subject detection (case-insensitive)
    subjects = [
        "Bengali", "Botany", "Chemistry", "Commerce", "Computer Science", "Economics",
        "Education", "Electronics", "English", "ENVS", "Film Studies", "History",
        "Journalism", "Mathematics", "Philosophy", "Physics", "Pol Science", "Zoology"
    ]
    for s in subjects:
        if s.lower() in text.lower():
            result["subject"] = s

    # Course type
    if "hons" in text.lower():
        result["course"] = "Hons"
    elif "gen" in text.lower():
        result["course"] = "Gen"

    return result

# Semantic similarity using SentenceTransformer
def semantic_ranking(query_text, metadata, top_k=10, threshold=0.3):
    q_emb = model.encode(query_text, convert_to_tensor=True)
    doc_texts = [
        " ".join([m["subject"], m["course"], m["semester_folder"], m["name"]])
        for m in metadata
    ]
    doc_emb = model.encode(doc_texts, convert_to_tensor=True)
    scores = util.cos_sim(q_emb, doc_emb).cpu().numpy()[0]

    ranked = []
    for i, score in enumerate(scores):
        if score >= threshold:
            ranked.append((metadata[i], score))

    ranked.sort(key=lambda x: x[1], reverse=True)
    return [r[0] for r in ranked[:top_k]]



Overwriting parse_query.py


In [53]:
%%writefile app.py
import streamlit as st
from gdrive_utils import get_drive_service, list_pdf_metadata, file_link
from parse_query import parse_query, semantic_ranking
import re

# 🎨 Streamlit page setup
st.set_page_config(page_title="📘 PYQ Chatbot", layout="wide")
st.markdown("""
    <h1 style="color:#3366cc;">📘 Previous Year Question Paper Chatbot</h1>
    <p style="font-size:18px;">🔍 Find question papers by subject, course, semester, year, or paper number — instantly!</p>
    <hr>
""", unsafe_allow_html=True)

# 📁 Your Google Drive root folder ID
folder_id = "1u14E3LRMkKlASQF-sjtkDrefjtq5ZWq9"

# Step 1: Welcome Screen and Initial Prompt
st.write("Welcome! 👋 Do you want to view or download Previous Year Question Papers (PYQs)?")
proceed = st.radio("Choose your action:", ('Yes, I want to see PYQs!', 'No, maybe later.'))

if proceed == 'Yes, I want to see PYQs!':
    # Step 2: Gather user's preferences
    col1, col2, col3 = st.columns(3)
    with col1:
        year = st.number_input("Enter the year:", min_value=2000, max_value=2100, step=1)
    with col2:
        subject = st.selectbox("Select the subject:", [
            "Bengali", "Botany", "Chemistry", "Commerce", "Computer Science", "Economics",
            "Education", "Electronics", "English", "ENVS", "Film Studies", "History",
            "Journalism", "Mathematics", "Philosophy", "Physics", "Pol Science", "Zoology"
        ])
    with col3:
        course = st.selectbox("Select the course type:", ["Hons", "Gen"])

    col4, col5 = st.columns(2)
    with col4:
        semester = st.selectbox("Select the semester:", [1, 2, 3, 4, 5, 6])
    with col5:
        # Extend paper number selection to 10
        paper_num = st.selectbox("Select paper number (optional):",
                                ["All papers"] + [f"Paper {i}" for i in range(1, 11)],
                                index=0)

    # Step 3: Perform Search
    if st.button("Search PYQs"):
        # Convert paper selection to pattern
        paper_pattern = ""
        if paper_num != "All papers":
            paper_index = int(paper_num.split(" ")[1])
            # Create pattern for exact matching
            paper_pattern = f"{course[0].lower()}" + ("" if paper_index == 1 else f"-{paper_index}")

        # Construct query
        query = f"{subject} {course} semester {semester} {year}"
        parsed = parse_query(query)

        # Load files from Google Drive
        try:
            service = get_drive_service()
            raw_meta = list_pdf_metadata(service, folder_id, year)

            if not raw_meta:
                st.warning("❌ No PDFs found for the specified year.")
                st.stop()

            meta = []
            for m in raw_meta:
                # Extract semester number
                sem_match = re.search(r'(\d+)', m["semester_folder"])
                m["sem"] = int(sem_match.group(1)) if sem_match else None

                # NEW: Precise paper pattern extraction
                # Look for pattern as a separate component in filename
                pattern = rf'([^a-zA-Z]{course[0].lower()}(?:-(\d+))?[^a-zA-Z])'
                paper_match = re.search(pattern, m["name"], re.IGNORECASE)

                if paper_match:
                    # Extract paper number
                    paper_num_str = paper_match.group(2)
                    if paper_num_str:
                        m["paper_num"] = int(paper_num_str)
                    else:
                        m["paper_num"] = 1

                    # Create pattern for filtering
                    if paper_num_str:
                        m["paper_pattern"] = f"{course[0].lower()}-{paper_num_str}"
                    else:
                        m["paper_pattern"] = course[0].lower()
                else:
                    # Default to paper 1 if pattern not found
                    m["paper_num"] = 1
                    m["paper_pattern"] = course[0].lower()

                meta.append(m)

            st.success(f"✅ Loaded {len(meta)} PDFs from Drive.")

            # Semantic matching
            ranked = semantic_ranking(query, meta, top_k=20)

            # Filtering
            filtered = []
            for m in ranked:
                # Apply filters
                if parsed["subject"] and parsed["subject"].lower() not in m["subject"].lower():
                    continue
                if parsed["course"] and parsed["course"].lower() not in m["course"].lower():
                    continue
                if parsed["sem"] and parsed["sem"] != m.get("sem"):
                    continue
                if paper_pattern:
                    # Exact match required for paper pattern
                    if m["paper_pattern"].lower() != paper_pattern.lower():
                        continue

                filtered.append(m)

            # Display results
            if not filtered:
                st.warning("❌ No matching question papers found.")
                st.info(f"Debug info: Year={year}, Subject={subject}, Course={course}, Semester={semester}, Paper={paper_pattern}")
                st.info(f"Top ranked PDF names: {[m['name'] for m in ranked[:5]]}")
            else:
                st.success(f"✅ Found {len(filtered)} matching papers:")
                for m in filtered:
                    file_id = m["id"]
                    download_url = file_link(file_id)
                    preview_url = f"https://drive.google.com/file/d/{file_id}/preview"

                    with st.expander(f"📄 {m['name']} (Paper {m['paper_num']})"):
                        st.markdown(f"""
                            <div style="background-color:#0a205a; padding:10px; border-radius:8px">
                                <b>📚 Subject:</b> {m['subject']} <br>
                                <b>🎓 Course:</b> {m['course']} <br>
                                <b>📅 Semester:</b> {m.get('sem')} <br>
                                <b>📝 Paper number:</b> {m['paper_num']} <br>
                                <b>📅 Year:</b> {m['year']} <br><br>
                                <a href="{download_url}" target="_blank" style="font-weight:bold; color:#3366cc;">📥 Download PDF</a>
                            </div>
                        """, unsafe_allow_html=True)

                        st.markdown(
                            f'<iframe src="{preview_url}" width="700" height="500" style="border:1px solid #ccc; border-radius:8px;" allow="autoplay"></iframe>',
                            unsafe_allow_html=True
                        )

        except Exception as e:
            st.error(f"❌ Error loading Drive files: {e}")
            st.stop()

else:
    st.write("Alright, feel free to come back later when you're ready to search for PYQs.")

# 👣 Footer
st.markdown("""
<hr>
<p style='font-size:14px; color:#888888;'>Built with ❤️ using Streamlit · Powered by Google Drive · NLP search enabled</p>
""", unsafe_allow_html=True)

Overwriting app.py


In [45]:
!pip install pyngrok
from pyngrok import ngrok
import threading
import time

# Launch Streamlit in a thread
def run_app():
    !streamlit run app.py

thread = threading.Thread(target=run_app)
thread.start()

# Wait a bit to let Streamlit start
time.sleep(5)
!ngrok config add-authtoken 2zHXBYbvnRZsFZ7p1voNLfEUxoU_49Tq3szQiEHY6owJ3CddQ
# Expose Streamlit app
public_url = ngrok.connect(8501)
print(f"🌐 Your app is live at: {public_url}")





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8507
  Network URL: http://172.28.0.12:8507
  External URL: http://34.74.176.200:8507

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🌐 Your app is live at: NgrokTunnel: "https://8caa-34-74-176-200.ngrok-free.app" -> "http://localhost:8501"
